In [1]:
import os

os.chdir(r'/Users/jialunlyu/Documents/GitHub/ProjectBuffalo/')

In [2]:
import buffalo.ingestion as ingestion
import buffalo.predictor as predictor

In [3]:
import torch
import pandas as pd

In [4]:
ingestor = ingestion.DataIngestion(ingestion.enum.API.ADVANTAGE)

In [5]:
ingestor.load_data(r'cached_data/ingestion.sqlite')

In [6]:
timestamps = ingestor.data['ADJUSTED_DAILY_STOCK'].query('symbol == "AAPL"').index

In [7]:
timestamps

DatetimeIndex(['2023-03-24', '2023-03-23', '2023-03-22', '2023-03-21',
               '2023-03-20', '2023-03-17', '2023-03-16', '2023-03-15',
               '2023-03-14', '2023-03-13',
               ...
               '1999-11-12', '1999-11-11', '1999-11-10', '1999-11-09',
               '1999-11-08', '1999-11-05', '1999-11-04', '1999-11-03',
               '1999-11-02', '1999-11-01'],
              dtype='datetime64[ns]', name='time', length=5887, freq=None)

In [8]:
predictor.util.predict_next_timestamps(timestamps, unit='D', num_indices=20)

Detected seasonality const,sin_0.20703125,sin_0.41015625,cos_0.41015625,sin_0.42578125,cos_0.42578125,sin_0.3828125,cos_0.3828125,sin_0.1875 from ts.


0    2023-03-25
1    2023-03-26
2    2023-03-27
3    2023-03-28
4    2023-03-29
5    2023-03-30
6    2023-03-31
7    2023-04-01
8    2023-04-02
9    2023-04-03
10   2023-04-04
11   2023-04-05
12   2023-04-06
13   2023-04-07
14   2023-04-08
15   2023-04-09
16   2023-04-10
17   2023-04-11
18   2023-04-12
19   2023-04-13
dtype: datetime64[ns]

In [ ]:
ingestor.data['ADJUSTED_DAILY_STOCK']

In [ ]:
predictor.util.detect_seasonality_timestamps(ingestor.data['ADJUSTED_DAILY_STOCK'].index, [1, 7, 14])

In [ ]:
stock_data = ingestor.data['ADJUSTED_DAILY_STOCK']
for k, v in ingestor.data.items():
    if k != 'ADJUSTED_DAILY_STOCK':
        stock_data = predictor.util.align_dataframe_by_time(stock_data, v)